# ARK Database Query Execution

Execute all predefined queries from queries.txt in a presentable format.


## Setup and Imports


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('Imports successful')


Imports successful


## Query Execution Function


In [2]:
def execute_query(title, query):
    try:
        df = pd.read_sql(query, engine)
        print(f'\n{"="*80}')
        print(f'✓ {title}')
        print(f'{"="*80}')
        print(f'Rows: {len(df)}\n')
        if len(df) > 0:
            display(df)
        else:
            print('[No results]')
        return df
    except Exception as e:
        print(f'\n✗ Error executing: {title}')
        print(f'Error: {e}')
        return None

print('✓ Query execution function ready')

✓ Query execution function ready


# ARK Database Query Execution

Execute all predefined queries from queries.txt in a presentable format.

## Setup and Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('✓ Imports successful')

✓ Imports successful


## Database Connection

In [ ]:
DB_CONFIG = {
    'host': 'localhost',
    'port': '15432',
    'database': 'ark_live_new_production',
    'username': '',
    'password': ''
}

connection_string = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

try:
    with engine.connect() as conn:
        result = conn.execute(text('SELECT current_database();'))
        db_name = result.fetchone()[0]
        print(f'✓ Connected to: {db_name}')
except Exception as e:
    print(f'✗ Connection failed: {e}')

✓ Connected to: ark_live_new_production


In [7]:
for grade in [2,3,4,5,6,7,8]:
    query = f"""
    SELECT
      q.id AS question_id,
      q.body AS question_text,
      a.id AS answer_id,
      a.body AS answer_text,
      tss.grade_level,
      tss.academic_year_id,
      AVG((tss.knowledge_score::decimal / NULLIF(tss.knowledge_total,0)) * 100) AS avg_knowledge_pct,
      COUNT(DISTINCT tss.id) AS student_count
    FROM questions q
    JOIN answers a ON a.question_id = q.id
    LEFT JOIN user_answers ua ON ua.answer_id = a.id
    LEFT JOIN testing_section_students tss ON tss.id = ua.testing_section_student_id
    LEFT JOIN users u ON u.id = tss.user_id AND u.role = 7
    WHERE (tss.subject_area_id = 1 OR tss.subject_area_id IS NULL)
      AND (tss.academic_year_id IN (2, 3, 4, 5) OR tss.academic_year_id IS NULL)
      AND tss.grade_level = {grade}
    GROUP BY q.id, q.body, a.id, a.body, tss.grade_level, tss.academic_year_id
    ORDER BY tss.academic_year_id, tss.grade_level, q.id, a.id;
    """

    df = execute_query(f"Grade {grade} Export", query)
    if df is not None and len(df) > 0:
        df.to_csv(f"dashboard_data_grade{grade}.csv", index=False)
        print(f"✅ Saved grade {grade} data ({len(df)} rows)")



✓ Grade 2 Export
Rows: 1062



,question_id,question_text,answer_id,answer_text,grade_level,academic_year_id,avg_knowledge_pct,student_count
0,2,Who is the Bread of Life?,1,Moses,2,2,41.216216,196
1,2,Who is the Bread of Life?,2,Jesus,2,2,62.076819,10037
2,2,Who is the Bread of Life?,3,Joseph,2,2,40.467985,149
3,2,Who is the Bread of Life?,4,Noah,2,2,40.509114,86
4,4,What takes place at Mass?,9,Bread and wine become the Body and Blood of Jesus,2,2,63.383716,8910
...,...,...,...,...,...,...,...,...
1057,7614,Which of the following do we have to do before...,31495,Read the Bible,2,5,60.763017,8981
1058,7615,"Who is a pastor, and what does he do?",31496,A priest who is in charge of a parish,2,5,69.078451,15920
1059,7615,"Who is a pastor, and what does he do?",31497,A deacon who assists a priest,2,5,62.801767,6857
1060,7615,"Who is a pastor, and what does he do?",31498,A religious brother who helps the pope,2,5,61.566593,7500


✅ Saved grade 2 data (1062 rows)

✓ Grade 3 Export
Rows: 1270



,question_id,question_text,answer_id,answer_text,grade_level,academic_year_id,avg_knowledge_pct,student_count
0,2,Who is the Bread of Life?,2,Jesus,3,2,76.000000,25
1,2,Who is the Bread of Life?,4,Noah,3,2,21.621622,1
2,4,What takes place at Mass?,9,Bread and wine become the Body and Blood of Jesus,3,2,76.238739,24
3,4,What takes place at Mass?,10,Bread and wine are shown as symbols for Jesus,3,2,45.945946,2
4,5,What is another phrase we use to describe the ...,13,The Head of God,3,2,65.494818,2660
...,...,...,...,...,...,...,...,...
1265,7616,"What can the phrase ""The Body of Christ"" descr...",31503,The Church,3,5,72.158361,10175
1266,7617,Which is NOT one of the four Marks of the Church?,31504,One,3,5,65.831346,16331
1267,7617,Which is NOT one of the four Marks of the Church?,31505,Holy,3,5,51.112018,1509
1268,7617,Which is NOT one of the four Marks of the Church?,31506,Credal,3,5,71.666718,15095


✅ Saved grade 3 data (1270 rows)

✓ Grade 4 Export
Rows: 1204



...

✅ Saved grade 6 data (1688 rows)

✓ Grade 7 Export
Rows: 1854



,question_id,question_text,answer_id,answer_text,grade_level,academic_year_id,avg_knowledge_pct,student_count
0,2,Who is the Bread of Life?,2,Jesus,7,2,87.309048,23
1,4,What takes place at Mass?,9,Bread and wine become the Body and Blood of Jesus,7,2,87.309048,23
2,28,Who shares the royal office of Christ as pries...,98,All baptized Christians,7,2,52.941176,1
3,28,Who shares the royal office of Christ as pries...,100,"Only bishops, priests, and deacons",7,2,60.784314,1
4,30,Which person of the Trinity was present at Pen...,105,The Father,7,2,56.862745,2
...,...,...,...,...,...,...,...,...
1849,7623,Which of the Ten Commandments prohibits adulte...,31531,The sixth commandment,7,5,61.249202,17233
1850,7624,Which of the following best describes what hap...,31532,The inclination to sin is removed,7,5,54.788884,14607
1851,7624,Which of the following best describes what hap...,31533,We die to sin and become new creations in Christ,7,5,60.736322,14884
1852,7624,Which of the following best describes what hap...,31534,We no longer need any other Sacraments,7,5,39.648318,1090


✅ Saved grade 7 data (1854 rows)

✓ Grade 8 Export
Rows: 1829



,question_id,question_text,answer_id,answer_text,grade_level,academic_year_id,avg_knowledge_pct,student_count
0,2,Who is the Bread of Life?,2,Jesus,8,2,91.891892,10
1,4,What takes place at Mass?,9,Bread and wine become the Body and Blood of Jesus,8,2,91.891892,10
2,31,Stories Jesus told are called:,1621,Parables,8,2,92.492492,9
3,31,Stories Jesus told are called:,1631,Gospels,8,2,86.486486,1
4,36,Who or what is made in the image and likeness ...,125,Human beings,8,2,92.492492,9
...,...,...,...,...,...,...,...,...
1824,7625,What three roles (or offices) of Jesus do all ...,31539,"Peacemaker, Mourner, and Victim",8,5,65.012223,8779
1825,7626,Can we get to heaven on our own? Why or why not?,31540,"Yes, because we have free will",8,5,59.265588,3234
1826,7626,Can we get to heaven on our own? Why or why not?,31541,"Yes, because we have the power to do good with...",8,5,55.668423,6464
1827,7626,Can we get to heaven on our own? Why or why not?,31542,"No, because we are forever stuck in a state of...",8,5,56.868341,4041


✅ Saved grade 8 data (1829 rows)


In [8]:
import glob

files = glob.glob("dashboard_data_grade*.csv")
combined = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

combined.to_csv("dashboard_data.csv", index=False)
print(f"✅ Combined {len(files)} files → {len(combined)} rows total")


✅ Combined 7 files → 10374 rows total


In [9]:
# --- Knowledge Dashboard (CSV-powered for Binder/Voila) ---

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Step 1: Load pre-fetched dataset (no DB connection) ---
df = pd.read_csv("dashboard_data.csv")

# --- Step 2: Validate and clean ---
if df is None or len(df) == 0:
    print("No data returned for this query.")
else:
    # Convert and clean columns
    df['question_id'] = df['question_id'].astype(str)
    df['answer_id'] = df['answer_id'].astype(str)
    df['grade_level'] = df['grade_level'].fillna('Unspecified').astype(str)
    df['academic_year_id'] = df['academic_year_id'].fillna(0).astype(int)
    df['avg_knowledge_pct'] = df['avg_knowledge_pct'].fillna(0)
    df['student_count'] = df['student_count'].fillna(0)

    # --- Map academic years ---
    year_labels = {
        2: '2021–2022',
        3: '2022–2023',
        4: '2023–2024',
        5: '2024–2025',
        0: 'Unspecified'
    }
    df['academic_year_label'] = df['academic_year_id'].map(year_labels).fillna('Unspecified')

    # --- Dropdown widgets ---
    dropdown_year = widgets.Dropdown(
        options=sorted([y for y in df['academic_year_label'].unique() if y != 'Unspecified'], reverse=True),
        value='2024–2025',
        description='Academic Year:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='45%')
    )

    dropdown_grade = widgets.Dropdown(
        options=sorted(df['grade_level'].unique().tolist()),
        value=sorted(df['grade_level'].unique().tolist())[0],
        description='Grade Level:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='30%')
    )

    # --- Function to populate questions based on year & grade ---
    def build_question_options(grade, year):
        subset = df[(df['grade_level'] == grade) & (df['academic_year_label'] == year)]
        questions = (
            subset[['question_id', 'question_text']]
            .drop_duplicates()
            .sort_values('question_text')
        )

        def truncate(text, n=90):
            return text if len(text) <= n else text[:n-3] + "..."

        return [(truncate(qt), qid) for qid, qt in zip(questions['question_id'], questions['question_text'])]

    question_dropdown = widgets.Dropdown(
        description='Question:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='90%')
    )
    question_dropdown.options = build_question_options(dropdown_grade.value, dropdown_year.value)
    question_dropdown.value = question_dropdown.options[0][1]

    # --- Output area for chart ---
    output_chart = widgets.Output()

    # --- Chart update logic ---
    def update_chart(*args):
        global fig
        with output_chart:
            output_chart.clear_output(wait=True)
            grade = dropdown_grade.value
            year = dropdown_year.value
            question_id = question_dropdown.value

            filtered = df[
                (df['grade_level'] == grade) &
                (df['academic_year_label'] == year) &
                (df['question_id'] == question_id)
            ]

            if filtered.empty:
                display(HTML("<p style='color:red;'>No data available for this selection.</p>"))
                return

            q_text = filtered['question_text'].iloc[0]
            display(HTML(f"<h4 style='margin-top:10px;'>{q_text}</h4>"))

            filtered = filtered.sort_values('avg_knowledge_pct', ascending=False)
            filtered['status'] = filtered['student_count'].apply(lambda x: 'Answered' if x > 0 else 'No Data')

            fig = px.bar(
                filtered,
                x='avg_knowledge_pct',
                y='answer_text',
                orientation='h',
                color='status',
                color_discrete_map={'Answered': '#4CAF50', 'No Data': '#CCCCCC'},
                text='avg_knowledge_pct',
                hover_data={
                    'answer_id': True,
                    'student_count': True,
                    'avg_knowledge_pct': ':.1f',
                    'answer_text': True
                },
                labels={'avg_knowledge_pct': 'Avg Knowledge %', 'answer_text': 'Answer'},
                height=550
            )

            fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
            fig.update_layout(
                title=f"Grade {grade} — {year}: Average Knowledge % by Answer",
                xaxis=dict(title='Average Knowledge %', range=[0, 100]),
                yaxis=dict(title='', automargin=True),
                plot_bgcolor='white',
                margin=dict(l=150, r=30, t=50, b=40),
                legend_title_text='Response Status'
            )

            fig.show()

    # --- Update question dropdown when filters change ---
    def on_filter_change(change):
        new_grade = dropdown_grade.value
        new_year = dropdown_year.value
        new_options = build_question_options(new_grade, new_year)
        if new_options:
            question_dropdown.options = new_options
            question_dropdown.value = new_options[0][1]
            update_chart()

    dropdown_year.observe(on_filter_change, names='value')
    dropdown_grade.observe(on_filter_change, names='value')
    question_dropdown.observe(update_chart, names='value')

    # --- Display the full dashboard ---
    ui = widgets.VBox([
        widgets.HBox([dropdown_year, dropdown_grade]),
        question_dropdown,
        output_chart
    ])

    display(ui)
    update_chart()
